In [2]:
# !pip install SimPEG

In [3]:
import numpy as np
import scipy.sparse as sp
from SimPEG import Mesh, Utils, Solver
from scipy.constants import mu_0, epsilon_0
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Sensitivity computuation for 1D magnetotelluric (MT)  problem

##  Purpose

With [SimPEG's](http://simpeg.xyz) mesh class, we descretize sensitivity function for a 1D magnetotelluric problem. Rather than generating full sensitivity matrix, we compute forwrad (`Jvec`) and adjoint (`Jtvec`) functional that can evalaute matrix-vector product. When computing senstivity functions, we also consider different forms of data: a) real and imaginary parts of impedance and b) apparaent resistivity. There are some milestones to be accomplished:

- Break apart senstivity function, $J_{\sigma}$ into to parts then derive each of them (using chain rules):

$$ 
J_{\sigma} = \frac{d P(u)}{d \sigma} = \frac{d P(u)}{d u} \frac{d u}{d \sigma}
$$

- Compute forward and adjoint sensitivity function: `Jvec` and `Jtvec`

- Test `Jvec` and `Jtvec`: Order test and Adjoint test

## Discretzation (forward simulation)

We define physical properties at cell centers, and stagger the electric and magnetic fields

- $\sigma$, $\mu$, $\epsilon$ : cell centers
- $E_x$: cell centers
- $H_y$: faces

<img src="./images/1DMT_discretize.png" width=200px> 

and use a finite difference approach to define the operators, this gives us the discrete system of equations

$$
\underbrace{
    \begin{bmatrix}
        \mathbf{Grad} & \imath \omega \mathbf{M}^{f}_{\mu} \\[0.3em]
        \mathbf{M}^{cc}_{\hat{\sigma}} & \mathbf{Div}           \\[0.3em]
    \end{bmatrix}
}_{\mathbf{A}}
\underbrace{
    \begin{bmatrix}
       \mathbf{e_x} \\[0.3em]
       \mathbf{h_y} \\[0.3em]
    \end{bmatrix}
}_{\mathbf{u}}
=
\underbrace{
    \begin{bmatrix}
       - \mathbf{B}\mathbf{e_x}^{BC} \\[0.3em]
       \boldsymbol{0} \\[0.3em]
    \end{bmatrix}
}_{\mathbf{rhs}}
$$

with 

- $\mathbf{e_x}$: Discrete $E_x$, on cell centers $[\text{nC} \times 1]$

- $\mathbf{h_y}$: Dicrete $H_x$, on cell faces $[(\text{nC}+1) \times 1]$

- $ \mathbf{Grad}$: Discrete gradient operator $[\text{nC} \times (\text{nC}+1)]$

- $ \mathbf{Div}$: Discrete divergence operator $[(\text{nC}+1) \times \text{nC}]$

- $\mathbf{M}^{f}_{\boldsymbol{\mu}} = \mathbf{diag}(\mathbf{Av^{cc2f}}  \boldsymbol{\mu})$ $[(\text{nC}+1) \times (\text{nC}+1)]$

- $\mathbf{M}^{cc}_{\boldsymbol{\hat{\sigma}}} = \mathbf{diag}(\boldsymbol{\hat{\sigma}})$ $[\text{nC} \times \text{nC}]$

- $\mathbf{B} \mathbf{e_x}^{BC}$ handles the boundary conditions

## What is data?

Measured data in general can be defined as:

$$ \mathbf{d} = P(\mathbf{u}) $$

where $P(\cdot)$ is a evaluation functional which takes a solution vector $\mathbf{u}$ ouputs data at a receiver location. 
We consdier two different type of data: 

- real and imaginary parts of the impedance ($Z_{xy}$)

- apparaent resistivity and phase

For the first case, we consider complex impedance ($Z_{xy}$) as our data, then $Z_{xy}$ can be evaluated from the solution vector $\mathbf{u}$:

$$ \mathbf{d} = Z_{xy} = - \mathbf{P}_{0}(1 / \mathbf{u}) $$

From complex-values $Z_{xy}$, we can compute real and imagniary part of the $Z_{xy}$ then the data can be defined as:

$$\mathbf{d} =  \begin{bmatrix}
       Re[Z_{xy}]  \\[0.3em]
       Im[Z_{xy}]  \\[0.3em]
     \end{bmatrix}$$

Similarly, for the second case the data can be evaluated from $Z_{xy}$. 


$$\mathbf{d} =  \begin{bmatrix}
       \rho_a \\[0.3em]
       \phi  \\[0.3em]
     \end{bmatrix}$$

where 

$$ \rho_a = \frac{1}{\mu_0\omega} |Z_{xy}|^2 $$

$$ \phi = \frac{180}{\pi}tan^{-1}(\frac{Im[Z_{xy}]}{Re[Z_{xy}]}) $$

## Sensitivity of datum with regard to $\sigma$:

Sensitivity function shows how much data is changed due to changes in model paramters. In general this is an important information to understand and design a geophysical survey. Further, to find a reasonable model we could use a gradient-based inversion technique, and for this case sensitivity function provide a gradient direction to find a solution. 

Sensitivity function can be defined as 

$$ J_{\sigma} = \frac{d P(u)}{d \sigma}$$

To obtain above sensitivity function in discrete space, we differentiate 

$$\mathbf{A}\mathbf{u} = \mathbf{rhs}$$

w.r.t ${\boldsymbol{\sigma}}$, which yields 

$$ \frac{d \mathbf{A}}{d {\boldsymbol{\sigma}}}\mathbf{u} +  \mathbf{A} \frac{d \mathbf{u} }{d {\boldsymbol{\sigma}}}=  0 $$

Rearranging and multiplyting $\mathbf{A}^{-1}$ in both sides yields

$$ \frac{d \mathbf{u} }{d {\boldsymbol{\sigma}}}=  -\mathbf{A}^{-1}\frac{d \mathbf{A}}{d {\boldsymbol{\sigma}}}\mathbf{u} $$

Consideration evaluation, $\mathbf{d} = P(\mathbf{u})$ we obtain

$$ \mathbf{J}_{{\boldsymbol{\sigma}}}  = 
\frac{\partial P(\mathbf{u})}{\partial {\mathbf{u}}}\Big(\frac{d \mathbf{u} }{d {\boldsymbol{\sigma}}}\Big) = 
-\frac{\partial P(\mathbf{u})}{\partial {\mathbf{u}}} \Big(\mathbf{A}^{-1}\frac{d \mathbf{A}}{d {\boldsymbol{\sigma}}}\mathbf{u}\Big) $$

Two derivatives need to be derived:

$$\frac{d \mathbf{A}}{d \boldsymbol{\sigma}}\mathbf{u}=?$$

$$\frac{\partial P(\mathbf{u})}{\partial \mathbf{u}}=?$$

The first one is straight forward having in mind that we are taking derivative vector ($\mathbf{A}\mathbf{u}$) with vector ($\boldsymbol{\sigma}$) yielding matrix:

$$ 
\frac{d \mathbf{A}}{d \boldsymbol{\sigma}}\mathbf{u} =
\mathbf{diag}\Big(
\begin{bmatrix}
\mathbf{0} & \mathbf{0} \\[0.3em]
\mathbf{I} & \mathbf{0}           \\[0.3em]
\end{bmatrix}
\begin{bmatrix}
       \mathbf{E}_x \\[0.3em]
       \mathbf{H}_y \\[0.3em]
    \end{bmatrix}
\Big)
= 
\mathbf{diag}\Big(
\begin{bmatrix}
    \mathbf{0} \\[0.3em]
       \mathbf{E}_x \\[0.3em]
    \end{bmatrix}
\Big)
$$

For the other one we consider when the data is defined as real and imaginary parts of $Z_{xy} = -\mathbf{P}_0\frac{1}{\mathbf{u}}$:

Taking derivative of $Z_{xy}$ w.r.t. $\mathbf{u}$ yields
$$
\frac{\partial Z_{
xy}}{\partial \mathbf{u}}=
\mathbf{P}_0\frac{1}{\mathbf{u}^2}
$$

$$ 
\frac{\partial P(\mathbf{u})}{\partial \mathbf{u}}=
\begin{bmatrix}
   \frac{\partial Re[Z_{xy}]}{\partial \mathbf{u}}  \\[0.3em]
   \frac{\partial Im[Z_{xy}]}{\partial \mathbf{u}}  \\[0.3em]
\end{bmatrix}  
=
\begin{bmatrix}
   Re[\mathbf{P}_0\frac{1}{\mathbf{u}^2}]  \\[0.3em]
   Im[\mathbf{P}_0\frac{1}{\mathbf{u}^2}]  \\[0.3em]
\end{bmatrix}          
$$


Now we can form sensitivity matrix $\mathbf{J}_{\sigma}$ by combining above equations:

$$ \mathbf{J}_{{\boldsymbol{\sigma}}}  = 
-\frac{\partial P(\mathbf{u})}{\partial {\mathbf{u}}} \Big(\mathbf{A}^{-1}\frac{d \mathbf{A}}{d {\boldsymbol{\sigma}}}\mathbf{u}\Big) $$

In addition, data can be apparent resistivity and phase, and we need corresponding $\frac{\partial P(\mathbf{u})}{\partial \mathbf{u}}$ for them.  We start with impedance in a polar form: 

$$Z_{xy} = |Z_{xy}|e^{\imath \phi}$$

Taking log yields

$$ log(Z_{xy}) = log(|Z_{xy}|) + \imath \phi$$

By taking derivative w.r.t $Z_{xy}$:

$$ \frac{1}{Z_{xy}} =\frac{1}{|Z_{xy}|} \frac{\partial |Z_{xy}|}{\partial Z_{xy}} + \imath \frac{\partial \phi}{\partial Z_{xy}}$$

Substituting $ \frac{\partial |Z_{xy}|}{\partial Z_{xy}} = \frac{Z_{xy}^{\\*}}{|Z_{xy}|}$ yields

$$ \frac{\partial \phi}{\partial Z_{xy}} = 0$$

which is an interesting result. For an apparent resistivity ($\rho_a = \frac{|\mathbf{Z}_{xy}^2|}{\mu_0\omega}$) we  obtain:

$$ 
\frac{\partial \rho_a}{\partial Z_{xy}} = \frac{\partial \rho_a}{\partial |Z_{xy}|}
\frac{\partial |Z_{xy}|}{\partial Z_{xy}}=
\frac{2 |Z_{xy}|}{\mu_0 \omega}
\frac{Z_{xy}^{\\*}}{|Z_{xy}|} = 
\frac{2 Z_{xy}^{\\*}}{\mu_0 \omega}
$$


## Computing sensitivity-vector product:

For 1D MT problem, sensitivity matrix ($N\times M$) can be small, hence generating sensitivity will not be a big deal. However, for instance any 3DEM problem, generating sensitivity matrix will require huge computatiion costs. To minimize that we only compute sensitivity-vector product. In forward case we compute:

$$ \mathbf{Jv} = \mathbf{J}_{\boldsymbol{\sigma}} \mathbf{v} $$


In [1]:
# This will be inputs
frequency = np.logspace(-3, 2, 25)
fmax, fmin = frequency.max(), frequency.min()
max_depth_core = 5000.
rho_half = 100.

NameError: name 'np' is not defined

In [4]:
print ("Smallest cell size = %d m") % (500*np.sqrt(rho_half/fmax) / 4.)
print ("Padding distance = %d m") % (500*np.sqrt(rho_half/fmin) * 2)
cs = 500*np.sqrt(rho_half/fmax) / 10.
skindepth2 = 500*np.sqrt(100/fmin) * 2

npad = 1
blength = cs*1.3**(np.arange(npad)+1)
while blength < skindepth2:
    npad+=1
    blength = (cs*1.3**(np.arange(npad)+1)).sum()

ncz = int(max_depth_core / cs)
hz = [(cs, npad, -1.3), (cs, ncz)]
mesh = Mesh.TensorMesh([hz], x0='N')
sigma = np.ones(mesh.nC) * 1./rho_half

Smallest cell size = 125 m
Padding distance = 316227 m


In [9]:
def dpred(sigma, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveCC2F * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)
    if dtype == "ri":
        return np.r_[Zxy.real, Zxy.imag]
    elif dtype == "rhopha":
        rhoa = abs(Zxy)**2 / (mu_0*omega)
        phase = np.rad2deg(np.arctan(Zxy.imag / Zxy.real))
        return np.r_[rhoa, phase]

def Jvec(sigma, v, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveCC2F * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)

    dAdsig_u_v = np.r_[np.zeros_like(Hy), Utils.sdiag(Ex)*v]
    dudsig_v = - (Ainv * (dAdsig_u_v))
    dZdsig_v = P0 * (Utils.sdiag(1./(u**2)) * dudsig_v)
    if dtype == "ri":
        dZrdsig_v = dZdsig_v.real
        dZidsig_v = dZdsig_v.imag
        return np.r_[dZrdsig_v, dZidsig_v]
    elif dtype == "rhopha":
        dZadZ = Zxy.conjugate() / abs(Zxy)
        drhodZa = 2. * abs(Zxy) / (mu_0*omega)
        drhodZ = drhodZa * dZadZ
        drhodsig_v = (drhodZ * dZdsig_v).real
#         dphadZ = 0.
#         dphadsig_v = (dphadZ * dZdsig_v).real
        return np.r_[drhodsig_v, 0.]

### Order test: Jvec

In [10]:
from SimPEG import Tests
def derChk(m):
    return [dpred(m, dtype="ri"), lambda mx: Jvec(m, mx, dtype="ri")]
Tests.checkDerivative(derChk, sigma, plotIt=False, num=3, eps=1e-20, dx=sigma*3)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.455e-02     7.603e-03      nan
 1   1.00e-02    4.122e-03     9.253e-05      1.915
 2   1.00e-03    4.205e-04     9.460e-07      1.990
========================= PASS! =========================
You are awesome.



True

In [11]:
from SimPEG import Tests
def derChk(m):
    return [dpred(m, dtype="rhopha"), lambda mx: Jvec(m, mx, dtype="rhopha")]
Tests.checkDerivative(derChk, sigma, plotIt=False, num=3, eps=1e-20, dx=sigma*3)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.308e+01     6.923e+00      nan
 1   1.00e-02    2.913e+00     8.748e-02      1.898
 2   1.00e-03    2.991e-01     9.925e-04      1.945
========================= PASS! =========================
That was easy!



True

## Computing Jtvec

Similarly, in adjoint case, we compute

$$ \mathbf{JTv} = \mathbf{J}_{\boldsymbol{\sigma}}^{T} \mathbf{v} $$

Computing $\mathbf{Jv}$ and $\mathbf{JTv}$ are straight forward from above derivation. 

$$ \mathbf{J}_{{\boldsymbol{\sigma}}}^T \mathbf{v} 
=  - (\frac{d \mathbf{A}}{d {\boldsymbol{\sigma}}}\mathbf{u} )^T
(\mathbf{A}^{T})^{-1} \frac{\partial P(\mathbf{u})}{\partial {\mathbf{u}}}^T \mathbf{v} $$

In [17]:
def misfit(sigma, dtype="ri", dobs=None):
    r = dpred(sigma, dtype=dtype) - dobs
    return 0.5 * np.linalg.norm(r)**2

def Jtvec(sigma, v, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveCC2F * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)    
    ATinv = Solver(A.T) # Factorize A matrix        
    
    if dtype == "ri":
        PTvr = (P0.T*np.r_[v[0]]).astype(complex)
        PTvi = P0.T*np.r_[v[1]]*-1j
        dZrduT_v = Utils.sdiag((1./(u**2)))*PTvr
        dZiduT_v = Utils.sdiag((1./(u**2)))*PTvi
        
        dAdsiguT = sp.hstack((Utils.spzeros(mesh.nC, mesh.nN), Utils.sdiag(Ex)))
        
        dZrdsigT_v = - (dAdsiguT*(ATinv*dZrduT_v)).real
        dZidsigT_v = - (dAdsiguT*(ATinv*dZiduT_v)).real        
        return dZrdsigT_v + dZidsigT_v
    
    elif dtype == "rhopha":
        dZadZ = Zxy / abs(Zxy)
        drhodZa = 2. * abs(Zxy) / (mu_0*omega)
        drhodZ = (drhodZa * dZadZ)
        drhodZT_v = drhodZ.conj() * np.r_[v[0]]        
        drhodsigT_v = Utils.sdiag((1./(u**2)))*(P0.T*drhodZT_v)
        dAdsiguT = sp.hstack((Utils.spzeros(mesh.nC, mesh.nN), Utils.sdiag(Ex)))
        drhodsigT_v = - (dAdsiguT*(ATinv*drhodsigT_v)).real
        
        return drhodsigT_v

In [18]:
sigma0 = sigma*3
dobs_ri = dpred(sigma, dtype="ri")
r = dpred(sigma0, dtype="ri") - dobs_ri 

Tests.checkDerivative(
    lambda m: [misfit(m, dobs=dobs_ri), Jtvec(m, r)],
    sigma0,
    plotIt=False,
    num=5
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    4.006e-03     8.689e-03      nan
 1   1.00e-02    5.141e-04     4.573e-05      2.279
 2   1.00e-03    4.725e-05     4.115e-07      2.046
 3   1.00e-04    4.687e-06     4.075e-09      2.004
 4   1.00e-05    4.684e-07     4.071e-11      2.000
========================= PASS! =========================
Go Test Go!



True

In [19]:
sigma0 = sigma*1.5
dobs_rhopha = dpred(sigma, dtype="rhopha")
r = dpred(sigma0, dtype="rhopha") - dobs_rhopha 

Tests.checkDerivative(
    lambda m: [misfit(m, dobs=dobs_rhopha, dtype="rhopha"), Jtvec(m, r, dtype="rhopha")],
    sigma0,
    plotIt=False,
    num=3, 
    dx = sigma0*2
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    4.321e+02     1.234e+01      nan
 1   1.00e-02    4.443e+01     1.681e-02      2.866
 2   1.00e-03    4.444e+00     1.204e-05      3.145
========================= PASS! =========================
That was easy!



True

## Adjoint tests

In [20]:
v = np.random.rand(mesh.nC)
w = np.random.rand(dobs_ri.shape[0])
wtJv = w.dot(Jvec(sigma0, v, dtype="ri"))
vtJtw = v.dot(Jtvec(sigma0, w, dtype="ri"))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 4.4408920985006262e-16, True)


In [21]:
v = np.random.rand(mesh.nC)
w = np.random.rand(dobs_rhopha.shape[0])
wtJv = w.dot(Jvec(sigma0, v, dtype="rhopha"))
vtJtw = v.dot(Jtvec(sigma0, w, dtype="rhopha"))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 2.2737367544323206e-13, True)
